> Descargando dataset


In [ ]:
!pip install kaggle # Instalando Kaggle CLI

from google.colab import files

uploaded = files.upload()  # Cargar API KEY

In [ ]:
# Moviendo API KEY a la carpeta de Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c online-sales -f TrainingDataset.csv -p /content # Descargando contenido

**Modelo Entrega Final Proyecto IA**


importamos lobrerias necesarias


In [556]:
import pandas as pd
import numpy as np

De sklearn importamos el modelo LinearRegression, en el cual especificaremos dos métodos: fit intercept=False porque después de varias pruebas se pudo apreciar una optimización en el tiempo de ejecución con este parámetro en False, ademas se le indico que positive=True para que solo nos devolviera valores positivos, ya que son ventas de un producto y no debería de darnos negativo nunca.


In [557]:
from sklearn.linear_model import LinearRegression


estimator = LinearRegression(fit_intercept=False, positive=True)

Abrimos los datos de trabajo


In [558]:
df = pd.read_csv("datos_procesados.csv")

Definimos nuestra función de relación, que nos da información sobre el peso e importancia de las columnas, esto nos servirá para terminar de purgar nuestra base de datos y no incluir las columnas que tengan una relación inferior a 10%.


In [559]:
def analisis_de_relacion(df, df1, df2, por):
    corrmat = df.corr()
    relacion = {}

    for i in df1:
        lista = []
        for j in df2:
            if corrmat[i][j] > por:
                lista.append(j)
        if lista:
            relacion[i] = lista
        return lista

Definimos el evaluador RMSLE (root mean square logarithmic error)


In [560]:
def accuracy(y_r, y_p):
    result = (sum((np.log(abs(y_p) + 1) - np.log(abs(y_r) + 1)) ** 2) / len(y_p)) ** 0.5

    return result

Definimos una función que va a partir la base de datos en dos, utilizaremos una parte de los datos para entrenar el modelo de regresión y la otra se utilizara para hacer una primera predicción, con la cual podremos evaluar que tan bien quedo el ajuste.


In [561]:
def split_fit_predict(estimator, X, y, train_pct):
    def split_data(X, y, pct):
        assert len(X) == len(y), "X and y must have the same length"
        assert pct > 0 and pct < 1, "pct must be in the (0,1) iterval"

        permutation = np.random.permutation(range(len(X)))
        per = permutation
        n1_elements = int(len(X) * pct)
        permutation_partition_1 = permutation[:n1_elements]
        permutation_partition_2 = permutation[n1_elements:]
        X1 = X[permutation_partition_1]
        X2 = X[permutation_partition_2]
        y1 = y[permutation_partition_1]
        y2 = y[permutation_partition_2]

        return X1, X2, y1, y2, per

    Xtr, Xts, ytr, yts, per = split_data(X, y, train_pct)

    estimator.fit(Xtr, ytr)
    preds_ts = estimator.predict(Xts)

    return (
        np.round(per),
        estimator,
        np.round(Xts),
        np.round(yts),
        accuracy(yts, preds_ts),
        np.round(preds_ts),
    )

En esta sección separaremos la base de datos original en diferentes grupos, X_p y y_p los utilizaremos para hacer una ultima predicción y comprobar la eficiencia del modelo, X y y se utilizaran para el entrenamiento.


In [562]:
df1 = df.iloc[:, :12]
df2 = df.iloc[:, 12:]

relacion = analisis_de_relacion(df, df1, df2, 0.1)

X_o = np.array(df[relacion])
y_o = np.array(df1)

por = 0.2
X_p = X_o[int(len(X_o) * por) :]
y_p = np.array(y_o[int(len(y_o) * por) :])

X = X_o[: int(len(X_o) * por)]
y = y_o[: int(len(y_o) * por)]

In [563]:
y_p

array([[ 3000.,  3000.,  2000., ...,   500.,  2000.,   500.],
       [35000., 11000.,  2000., ...,  2000.,  2000.,  2000.],
       [10000.,  3000.,  3000., ...,  2000.,  2000.,  2000.],
       ...,
       [10000.,  2000.,   500., ...,   500.,  2000.,   500.],
       [11000.,  3000.,  2000., ...,  2000.,   500.,   500.],
       [18000.,  6000.,  2000., ...,  2000.,  2000.,  3000.]])

En esta sección hacemos un ciclo, en el cual aplicamos el método creado anteriormente, repitiéndolo hasta encontrar un valor aceptable, este valor se tomo analizando las soluciones, se encontró que puede tener valores inferiores a 0.82, pero lo mejor para la optimización en tiempo es dejarlo en 0.83.


In [564]:
c = 0


while True:
    prediccion = split_fit_predict(estimator, X, y, 0.2)

    c += 1

    if np.mean(prediccion[-2]) < 0.83:
        break

Se extrae el estimator entrenado


In [565]:
estimator_entrenado = prediccion[1]

Hacemos predicciones tanto a la base de datos original como a la que separamos y medimos su error.


In [566]:
pred = np.round(estimator_entrenado.predict(X_p))

error = accuracy(y_p, pred)

predT = np.round(estimator_entrenado.predict(X_o))

errorT = accuracy(y_o, predT)

como podemos observar el error es muy bajo en algunas predicciones llegando en ocasiones a valores inferiores a 0.7 para algunas columnas.


In [567]:
errorT, error

(array([0.95027117, 1.09752583, 1.00365922, 0.99947819, 0.92818242,
        0.81440812, 0.74950759, 0.7021851 , 0.85772613, 0.80585501,
        0.62590658, 0.59614803]),
 array([0.96968636, 1.11070876, 1.01558873, 1.02504867, 0.9549249 ,
        0.82857594, 0.76759381, 0.71526858, 0.86591562, 0.80384771,
        0.64064875, 0.60792029]))